# Optimization Hw5
## 1
So far we have learned two ways to deal with linear programming problems. One way is using the simplex algorithm, and the other is using
interior point methods. Pick any two LP problems. Perform optimization
with both kinds of approaches. For each test problem, comment on teh accuracy and convergence speed of the two approaches.

The first problem we are going to optimize is 3.45 from the book by Rao.
$$f=240x_1 + 104x_2 + 60x_3 + 19x_4$$
s.t.
$$20x_1 + 9x_2 + 6x_3 + x_4\leq 20$$
$$10x_1 + 4x_2 + 2x_3 + x_4\leq 10$$
$$x_i \geq 0, i=1,\dots,4$$

[Source](https://radzion.com/blog/operations/simplex) for the simplex algorithm.

In [1]:
import numpy as np
import math
import numba

In [2]:
def simplex(c, A, b):
    iterations = 0
    table = to_tableau(c, A, b)
    while not_solved(table):
        iterations += 1
        pivot_position = get_pivot(table)
        table = pivot_step(table, pivot_position)
    return (get_solution(table), iterations)
def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

# Check if this is the best solution
def not_solved(table):
    z = table[-1]
    return any( x > 0 for x in z[:-1])
def get_pivot(table):
    z = table[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
     
    restrictions = []
    for eq in table[:-1] :
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)
    row = restrictions.index(min(restrictions))
    return row, column

def pivot_step(table, pivot_position):
    new_table = [[] for eq in table]
    i, j = pivot_position
    pivot_value = table[i][j]
    new_table[i] = np.array(table[i]) / pivot_value 
    
    for eq_i, eq in enumerate(table):
        if eq_i != i:
            multiplier = np.array(new_table[i]) * table[eq_i][j]
            new_table[eq_i] = np.array(table[eq_i]) - multiplier
    return new_table

def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) -1

def get_solution(table):
    columns = np.array(table).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
    return solutions

### Prime Affine Scaling algorithm
[Source](https://computsimu.blogspot.com/2015/07/interior-point-method-primal-affine.html)
[Source](https://github.com/manymuch/AffineScaling)

In [7]:
from HelperFunction import LinProgBaseClass, StandardFormTransformer


class PAS(LinProgBaseClass):

    # epsilon is optimality threshold
    # beta is the stepsize that controling the elipsoid size
    def __init__(self, A, b, c, x, epsilon=1e-2, beta=0.1, trace=False):
        super(PAS, self).__init__(A, b, c, x, trace=trace)

        if beta <= 0 or beta >= 1:
            raise RuntimeError("beta must between (0,1)")
        if epsilon <= 0:
            raise RuntimeError("epsilon must be positive")

        self.X_k = np.diag(x[:, 0])
        self.p_k = None
        self.r_k = None
        self.epsilon = epsilon  # optimality threshold
        self.beta = beta

    # reached optimal: return true
    def __OptimalityCheck(self):
        distance = np.sum(self.X_k @ self.r_k)
        #  dual feasibliity and optimality
        return distance > 0 and distance < self.epsilon

    # unbounded: return true
    def __UnboundenessCheck(self):
        reduced_cost = - self.X_k @ self.X_k @ self.r_k
        return np.all(np.greater_equal(reduced_cost, 0))

    def __Caculate_r(self):
        self.p_k = np.linalg.inv(self.A @ self.X_k @ self.X_k @ np.transpose(
            self.A)) @ self.A @ self.X_k @ self.X_k @ self.c
        self.r_k = self.c - np.transpose(self.A) @ self.p_k

    def __Update_X(self):
        move = self.X_k @ self.X_k @ self.r_k / \
            self.__gamma(self.X_k @ self.r_k)
        move = np.diag(move[:, 0])
        self.X_k = self.X_k - self.beta * move
    
    def __gamma(self, input):
        clipped = np.clip(input, 0, None)
        return np.max(clipped)

    def Run(self):
        self.__Caculate_r()
        iterations = 0
        while not self.__OptimalityCheck():
            iterations += 1
            if self.__UnboundenessCheck():
                print("the input LP problem is unbounded")
                return None
            self.__Update_X()
            self.__Caculate_r()
            if self.trace:
                expanded_X = np.expand_dims(np.diag(self.X_k),axis=0)
                self.traces = np.concatenate((self.traces,expanded_X),axis=0)
        return np.diag(self.X_k), iterations
    
    def GetTraces(self):
        return self.traces



In [8]:
# Using simplex method
c = [240, 104, 60, 19, 0, 0]
A = [
    [20, 9, 6, 1, 1, 0],
    [10, 4, 2, 1, 0, 1]
]
b = [20, 10]
solution, iterations = simplex(c, A, b)
print(solution, iterations)

[0, 2.0, 0, 2.0, 0, 0] 3


In [11]:
c = np.array([240, 104, 60, 19, ]).reshape((1,4)).T
b = np.array(b).reshape((2, 1))
A = np.array([
    [20, 9, 6, 1],
    [10, 4, 2, 1]
])

x_origin = np.asarray([[0.5, 0.5, 0.5, 0.5]]).T

(Anew, _, cnew, xnew) = StandardFormTransformer(A, b, c, x_origin)
print(Anew.shape, b.shape, cnew.shape, xnew.shape)
solver = PAS(Anew, b, cnew, xnew, trace=True)
sol, iterations = solver.Run()

print(f'Solution {sol} found in {iterations} iterations')

(2, 6) (2, 1) (6, 1) (6, 1)
True
Solution [9.68162989e-06 2.23420424e-05 3.87259177e-05 1.22282500e-04
 1.99992507e+01 9.99961408e+00] found in 103 iterations


## 1b
We apply to problem 3.14 in the textbook
Maximize $$f = 19x + 7y$$
subject to 
$$7x + 6y \leq 42$$
$$5x + 9y \leq 45$$
$$x - y \leq 4$$
$$x\geq 0, y\geq 0$$

In [15]:
c = [-19, 7, 0, 0, 0, 0]
A = [
    [7, 6, 1, 0, 0, 0],
    [5, 9, 0, 1, 0, 0],
    [1, -1, 0, 0, 1, 0]
]
b = [42, 45, 4]
solution, iterations = simplex(c, A, b)
print(f'Solution {solution} found in {iterations} iterations')

Solution [0, 5.0, 12.0, 0, 9.0, 0] found in 1 iterations


In [21]:
c = np.array([-19, 7]).reshape((1, 2)).T
b = np.array(b).reshape((3, 1))
A = np.array([
    [7, 6 ],
    [5, 9 ],
    [1, -1]
])
x_origin = np.array([[0.5, 0.5]]).T
(Anew, _, cnew, xnew) = StandardFormTransformer(A, b, c, x_origin)
solver = PAS(Anew, b, cnew, xnew, trace=True)
sol, iterations = solver.Run()
print(f'Solution {sol} found in {iterations} iterations')

True
Solution [5.07637133e+00 1.07673736e+00 4.97648295e-03 9.92750706e+00
 3.66029409e-04] found in 98 iterations


## 2
So far we have learned two ways to deal with constrained nonlinear optimizatino problems. One way is the indirect approach (e.g. penalty functions) and the other is the direct approach (e.g. Sequential Quadratic Programming, SQP). Pick any two test functions (neither linear nor quadratic functions) with constraints. Perform optimization with both direct and indirect approaches.

For the first function, we can estimate $$f(x, y) = \sin(x + y)$$
with the constraints
$$x + y \leq 2$$
$$x + y \geq -2$$

In [36]:
from scipy.optimize import minimize, rosen
import math
def f(x:np.array):
    return math.sin(x[0] + x[1])

In [38]:
# SQP
bounds = [(-2, 2), (-2, 2)] # Bounds for x and y
x = np.array([0, 0])
minimizer = minimize(f, x, method='SLSQP', bounds=bounds)
print(minimizer)

     fun: -0.9999999999180504
     jac: array([1.2807548e-05, 1.2807548e-05])
 message: 'Optimization terminated successfully'
    nfev: 15
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.78539176, -0.78539176])


In [69]:
# Interior penalty methods
x_c = np.array([0, 0])
h1 = lambda x: (x[0] + x[1] - 2)
h2 = lambda x: (x[0] + x[1] + 2)

i = 1
constraints = ({'type': 'ineq', 'fun': h1},
               {'type': 'ineq', 'fun': h2}
    )
while i < 100:
    xprev = x_c
    i += 1
    L = lambda x: f(x) + min(0, h1(x)) ** 2 + min(0, h2(x)) ** 2
    x_c = minimize(f, x_c, constraints=constraints).x
    
    if np.array_equal(xprev, x_c):
        break
        
print(x_c, i)

[2.35611271 2.35611271] 3


## 2b
For the second equation we are going to maximize
$$f(x,y) = x\sin(x + \sin(y))$$
subject to
$$x + y \leq 1$$

In [70]:
## SQP
def fnc(x:np.array):
    return -x[0] * math.sin(x[0] + math.sin(x[1])) 
h1 = lambda x: (x[0] + x[1]  - 1)
constraints = (
                {'type': 'ineq', 'fun': h1}
            )
x = np.array([1, 0])
minimizer = minimize(fnc, x, method='SLSQP', constraints=constraints)
print(minimizer)

     fun: -2.7442815484609415
     jac: array([-0.00011867,  0.00016609])
 message: 'Optimization terminated successfully'
    nfev: 33
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([ 2.90254668, -1.57062065])


In [71]:
## Interior point
i = 0
x = np.array([1, 0])
while i < 100:
    xprev = x
    L = lambda x: fnc(x) + min(0, h1(x)) ** 2
    sol = minimize(L, x_c, constraints=constraints)
    if np.array_equal(sol.x, xprev):
        break
    i += 1
print(sol)
print(f"Solution {sol.x} found in {i} iterations")

     fun: -2.744281511861129
     jac: array([ 0.00032538, -0.00026703])
 message: 'Optimization terminated successfully'
    nfev: 31
     nit: 10
    njev: 10
  status: 0
 success: True
       x: array([2.90267744, 4.71210667])
Solution [2.90267744 4.71210667] found in 100 iterations
